# Análise Exploratória de Dados - Sistema de Prevenção de Fraudes

Este notebook apresenta uma análise exploratória completa dos dados transacionais para detecção de fraudes.

## Objetivos
- Entender a distribuição dos dados
- Identificar padrões de fraude
- Detectar anomalias
- Gerar insights para modelagem

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import sys
import os

# Adiciona o diretório raiz ao path
sys.path.append('..')

# Importações do projeto
from src.data.data_loader import DataLoader
from src.data.exploratory_analysis import ExploratoryDataAnalysis
from src.utils.helpers import format_currency, format_percentage

# Configurações
warnings.filterwarnings('ignore')
try:
    plt.style.use('seaborn-v0_8')
except:
    plt.style.use('default')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Bibliotecas importadas com sucesso!")

## Carregamento dos Dados

In [ ]:
# Carrega dados
loader = DataLoader()
users_df, transactions_df = loader.load_synthetic_data()

print(f"Dados carregados:")
print(f"   - Usuários: {len(users_df):,}")
print(f"   - Transações: {len(transactions_df):,}")
print(f"   - Período: {transactions_df['timestamp'].min()} a {transactions_df['timestamp'].max()}")

## Visão Geral dos Dados

In [ ]:
# Informações básicas sobre transações
print("Informações sobre Transações:")
print(transactions_df.info())

print("\nEstatísticas Descritivas:")
print(transactions_df.describe())

In [ ]:
# Análise de fraudes
fraud_stats = {
    'total_transactions': len(transactions_df),
    'fraud_transactions': transactions_df['is_fraud'].sum(),
    'fraud_rate': transactions_df['is_fraud'].mean(),
    'total_volume': transactions_df['amount'].sum(),
    'fraud_volume': transactions_df[transactions_df['is_fraud']]['amount'].sum()
}

fraud_stats['fraud_volume_rate'] = fraud_stats['fraud_volume'] / fraud_stats['total_volume']

print("Estatísticas de Fraude:")
print(f"   - Total de transações: {fraud_stats['total_transactions']:,}")
print(f"   - Transações fraudulentas: {fraud_stats['fraud_transactions']:,}")
print(f"   - Taxa de fraude: {format_percentage(fraud_stats['fraud_rate'])}")
print(f"   - Volume total: {format_currency(fraud_stats['total_volume'])}")
print(f"   - Volume de fraude: {format_currency(fraud_stats['fraud_volume'])}")
print(f"   - Taxa de volume fraudulento: {format_percentage(fraud_stats['fraud_volume_rate'])}")

## Análise de Distribuições

In [ ]:
# Distribuição de valores
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histograma geral
axes[0, 0].hist(transactions_df['amount'], bins=50, alpha=0.7, color='blue')
axes[0, 0].set_title('Distribuição de Valores - Todas as Transações')
axes[0, 0].set_xlabel('Valor (R$)')
axes[0, 0].set_ylabel('Frequência')

# Histograma por tipo
fraud_amounts = transactions_df[transactions_df['is_fraud']]['amount']
legit_amounts = transactions_df[~transactions_df['is_fraud']]['amount']

axes[0, 1].hist(legit_amounts, bins=50, alpha=0.7, label='Legítimas', color='green')
axes[0, 1].hist(fraud_amounts, bins=50, alpha=0.7, label='Fraudes', color='red')
axes[0, 1].set_title('Distribuição de Valores por Tipo')
axes[0, 1].set_xlabel('Valor (R$)')
axes[0, 1].set_ylabel('Frequência')
axes[0, 1].legend()

# Box plot
transactions_df.boxplot(column='amount', by='is_fraud', ax=axes[1, 0])
axes[1, 0].set_title('Box Plot - Valores por Tipo')
axes[1, 0].set_xlabel('É Fraude?')
axes[1, 0].set_ylabel('Valor (R$)')

# Distribuição por hora
hourly_counts = transactions_df.groupby(['transaction_hour', 'is_fraud']).size().unstack(fill_value=0)
hourly_counts.plot(kind='bar', ax=axes[1, 1], color=['green', 'red'])
axes[1, 1].set_title('Transações por Hora do Dia')
axes[1, 1].set_xlabel('Hora')
axes[1, 1].set_ylabel('Número de Transações')
axes[1, 1].legend(['Legítimas', 'Fraudes'])
axes[1, 1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## Análise Temporal

In [ ]:
# Análise por hora do dia
hourly_fraud_rate = transactions_df.groupby('transaction_hour')['is_fraud'].agg(['count', 'sum', 'mean']).reset_index()
hourly_fraud_rate.columns = ['hour', 'total_transactions', 'fraud_count', 'fraud_rate']

# Gráfico interativo
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Volume de Transações por Hora', 'Taxa de Fraude por Hora'),
    specs=[[{"secondary_y": False}], [{"secondary_y": False}]]
)

# Volume por hora
fig.add_trace(
    go.Bar(x=hourly_fraud_rate['hour'], y=hourly_fraud_rate['total_transactions'], 
           name='Total de Transações', marker_color='lightblue'),
    row=1, col=1
)

# Taxa de fraude por hora
fig.add_trace(
    go.Scatter(x=hourly_fraud_rate['hour'], y=hourly_fraud_rate['fraud_rate'], 
               mode='lines+markers', name='Taxa de Fraude', line=dict(color='red')),
    row=2, col=1
)

fig.update_layout(height=600, title_text="Análise Temporal - Transações por Hora")
fig.update_xaxes(title_text="Hora do Dia", row=2, col=1)
fig.update_yaxes(title_text="Número de Transações", row=1, col=1)
fig.update_yaxes(title_text="Taxa de Fraude", row=2, col=1, tickformat='.2%')

fig.show()

print("⏰ Insights Temporais:")
print(f"   - Hora com mais fraudes: {hourly_fraud_rate.loc[hourly_fraud_rate['fraud_count'].idxmax(), 'hour']}h")
print(f"   - Hora com maior taxa de fraude: {hourly_fraud_rate.loc[hourly_fraud_rate['fraud_rate'].idxmax(), 'hour']}h")
print(f"   - Taxa média de fraude: {hourly_fraud_rate['fraud_rate'].mean():.2%}")

## Análise por Categoria de Comerciante

In [ ]:
# Análise por categoria
category_analysis = transactions_df.groupby('merchant_category').agg({
    'amount': ['count', 'sum', 'mean'],
    'is_fraud': ['sum', 'mean']
}).round(4)

category_analysis.columns = ['total_transactions', 'total_volume', 'avg_amount', 'fraud_count', 'fraud_rate']
category_analysis = category_analysis.sort_values('fraud_rate', ascending=False)

print("Análise por Categoria de Comerciante:")
print(category_analysis)

# Gráfico de taxa de fraude por categoria
fig = px.bar(
    x=category_analysis.index,
    y=category_analysis['fraud_rate'],
    title='Taxa de Fraude por Categoria de Comerciante',
    labels={'x': 'Categoria', 'y': 'Taxa de Fraude'},
    color=category_analysis['fraud_rate'],
    color_continuous_scale='Reds'
)
fig.update_xaxes(tickangle=45)
fig.update_yaxes(tickformat='.2%')
fig.show()

print(f"\nTop 3 categorias com maior taxa de fraude:")
for i, (category, row) in enumerate(category_analysis.head(3).iterrows()):
    print(f"   {i+1}. {category}: {row['fraud_rate']:.2%} ({row['fraud_count']:.0f} fraudes)")

## Análise por Método de Pagamento

In [ ]:
# Análise por método de pagamento
payment_analysis = transactions_df.groupby('payment_method').agg({
    'amount': ['count', 'sum', 'mean'],
    'is_fraud': ['sum', 'mean']
}).round(4)

payment_analysis.columns = ['total_transactions', 'total_volume', 'avg_amount', 'fraud_count', 'fraud_rate']
payment_analysis = payment_analysis.sort_values('fraud_rate', ascending=False)

# Gráficos
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Distribuição de métodos de pagamento
payment_counts = transactions_df['payment_method'].value_counts()
axes[0].pie(payment_counts.values, labels=payment_counts.index, autopct='%1.1f%%')
axes[0].set_title('Distribuição por Método de Pagamento')

# Taxa de fraude por método
payment_analysis['fraud_rate'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Taxa de Fraude por Método de Pagamento')
axes[1].set_ylabel('Taxa de Fraude')
axes[1].tick_params(axis='x', rotation=45)
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))

plt.tight_layout()
plt.show()

print("Análise por Método de Pagamento:")
print(payment_analysis)

## Análise Geográfica

In [ ]:
# Análise por país
country_analysis = transactions_df.groupby('country').agg({
    'amount': ['count', 'sum'],
    'is_fraud': ['sum', 'mean']
}).round(4)

country_analysis.columns = ['total_transactions', 'total_volume', 'fraud_count', 'fraud_rate']
country_analysis = country_analysis.sort_values('fraud_rate', ascending=False)

print("Análise por País:")
print(country_analysis)

# Gráfico de países com maior risco
top_risk_countries = country_analysis.head(10)

fig = px.bar(
    x=top_risk_countries.index,
    y=top_risk_countries['fraud_rate'],
    title='Taxa de Fraude por País (Top 10)',
    labels={'x': 'País', 'y': 'Taxa de Fraude'},
    color=top_risk_countries['fraud_rate'],
    color_continuous_scale='Reds'
)
fig.update_yaxes(tickformat='.2%')
fig.show()

## Detecção de Anomalias

In [ ]:
# Executa análise completa usando o módulo EDA
eda = ExploratoryDataAnalysis()

# Gera análises
overview = eda.generate_data_overview(transactions_df)
patterns = eda.analyze_fraud_patterns(transactions_df)
anomalies = eda.detect_anomalies(transactions_df)

print("Anomalias Detectadas:")
print(f"   - Outliers por IQR: {anomalies['amount_anomalies']['outliers_iqr_count']:,} ({anomalies['amount_anomalies']['outliers_iqr_percentage']:.2%})")
print(f"   - Outliers por Z-score: {anomalies['amount_anomalies']['outliers_zscore_count']:,} ({anomalies['amount_anomalies']['outliers_zscore_percentage']:.2%})")
print(f"   - Valores redondos suspeitos: {anomalies['amount_anomalies']['suspicious_round_amounts']:,}")
print(f"   - Transações noturnas: {anomalies['temporal_anomalies']['night_transactions']:,} ({anomalies['temporal_anomalies']['night_percentage']:.2%})")

if 'velocity_anomalies' in anomalies:
    print(f"   - Usuários de alta velocidade: {anomalies['velocity_anomalies']['high_velocity_users']:,}")

## Matriz de Correlação

In [ ]:
# Seleciona features numéricas para correlação
numeric_features = transactions_df.select_dtypes(include=[np.number]).columns
correlation_matrix = transactions_df[numeric_features].corr()

# Heatmap de correlação
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Matriz de Correlação - Features Numéricas')
plt.tight_layout()
plt.show()

# Correlações mais fortes com fraude
fraud_correlations = correlation_matrix['is_fraud'].abs().sort_values(ascending=False)
print("Features mais correlacionadas com fraude:")
for feature, corr in fraud_correlations.head(10).items():
    if feature != 'is_fraud':
        print(f"   - {feature}: {corr:.4f}")

## Conclusões e Insights

### Principais Descobertas:

1. **Taxa de Fraude**: A taxa geral de fraude é de aproximadamente 2%
2. **Padrões Temporais**: Certas horas do dia apresentam maior risco
3. **Categorias de Risco**: Algumas categorias de comerciantes são mais propensas a fraudes
4. **Métodos de Pagamento**: Diferentes métodos apresentam riscos distintos
5. **Anomalias**: Identificadas várias anomalias que podem indicar fraudes

### Próximos Passos:

1. **Feature Engineering**: Criar novas features baseadas nos insights
2. **Modelagem**: Desenvolver modelos de ML para detecção automática
3. **Regras de Negócio**: Implementar regras baseadas nos padrões identificados
4. **Monitoramento**: Estabelecer alertas para anomalias em tempo real